In [3]:
import requests
from bs4 import BeautifulSoup
import json

In [4]:
from model import User

In [5]:
def diagnostico_cat():
    url_diagnostico = "https://cpal.edu.pe/diagnostico-y-tratamiento/diagnostico/"
    cpal_diagnostico = requests.get(url_diagnostico)   
    s_cpal = BeautifulSoup(cpal_diagnostico.text, 'lxml') 
    secciones = s_cpal.find_all('a', attrs={'class':'b37-linkcategoria'})
    a = []
    for seccion in secciones:
        a.append(seccion.text)
    return a

In [6]:
diagnostico_cat()

['Lenguaje', 'Aprendizaje', 'Habla', 'Audición', 'Psicomotricidad', 'UDAD']

In [7]:
dict_func = {
    "diagnostico":diagnostico_cat()
}

In [8]:
dict_func['diagnostico']

['Lenguaje', 'Aprendizaje', 'Habla', 'Audición', 'Psicomotricidad', 'UDAD']

In [9]:
data = json.loads(open('./01_mvp.json',mode="r",encoding="utf-8").read())

In [10]:
for i in data['intents']:
    print(i.keys())

dict_keys(['tag', 'patterns', 'responses'])
dict_keys(['tag', 'patterns', 'responses'])


In [11]:
def json_scan(message:str) -> str:
    for i in data['intents']:
        if message in i['patterns']:
            return {'response': i['responses'],
                    'tag': i['tag']}
json_scan(str(2))['response']

'function'

In [12]:
message = str(2)
def val_message(message:str):
    r = json_scan(message)
    if r['response'] != "function":
        return r['response']
    else:
        return str(dict_func[r['tag']]) 

val_message(message)  
    

"['Lenguaje', 'Aprendizaje', 'Habla', 'Audición', 'Psicomotricidad', 'UDAD']"

In [57]:
import base64
 
with open("./data/gatitos.jpg", "rb") as imageFile:
    data = base64.b64encode(imageFile.read())


In [51]:
with open("data/tiny.JPG", "rb") as img:
    data = base64.b64encode(img.read())


In [49]:
type(str.encode(str(data)[2:-1]))

bytes

In [58]:
str_data =data.decode("utf-8") 

In [22]:
my_str = "hello world"
my_str_as_bytes = str.encode(my_str)
print(my_str_as_bytes) # ensure it is byte representation
my_decoded_str = my_str_as_bytes.decode()
print(my_decoded_str)

b'hello world'
hello world


In [55]:


decodeit = open('gatitos.jpeg', 'wb')
decodeit.write(base64.b64decode((data)))
decodeit.close()

In [ ]:
'%d-%m-%Y'

In [56]:
from datetime import datetime

def get_byte_code(user:dict):
    path = f'{user["user_id"]}-{datetime.now().date().strftime("%d-%m-%Y")}.jpeg'
    decodeit = open(path, 'wb')
    data = base64.b64decode(str.encode(user["message"]))
    decodeit.write(data)
    decodeit.close()
    return  path
    
    

In [40]:
def byte_code_to_image(path:str) -> str:
    with open(path, "rb") as imageFile:
        data = base64.b64encode(imageFile.read())
    return data

In [59]:
user = {
    "user_id":"gatito",
    "message":str_data
}

In [ ]:
path  = get_byte_code(user)
byte_code_to_image(path)

## Bot Respuesta


In [1]:
from pydantic import BaseModel, Field

class User(BaseModel):
    user_id : str = Field(
        ...,
        min_length=1,
        max_length=50,
        example='51952841852@.se'
    )
    message : str = Field(
        ...,
        example='/proceso de seleccion'
    )

In [101]:
masami = User
xiomara =User

masami.user_id = "51952841852@c.us"
xiomara.user_id = "51934935843@c.us"

In [3]:
import json
data = json.loads(open('./respuesta.json',mode="r",encoding="utf-8").read())

In [4]:
dic={}
def historial(user:User,reset=False):
    global cuenta
    cuenta += 1
    dic[user.user_id] = cuenta
    if reset:
        cuenta = 0
    return dic

In [6]:
class Historial:
    dic= {}

    def add_user(self,user:User) -> dict:
        dic[user.user_id] = [0,""]
        return dic[user.user_id]

    def count(self,user:User,direccion:str = "" ,count=True,reset:bool =False) -> dict: 
        if count:
            dic[user.user_id][0] += 1
        dic[user.user_id][1] += direccion
        if reset:
            dic[user.user_id][0] = 0
            dic[user.user_id][1] = ""
        return dic
    
    def get_count(self, user:User) -> dict:
        return dic[user.user_id]

In [7]:
dic['51934935842@c.us'] = [0,""]

In [8]:
dic['51934935842@c.us'][0] += 1
dic['51934935842@c.us'][1] += "r"
dic

{'51934935842@c.us': [1, 'r']}

In [9]:
hist = Historial()

In [103]:
xiomara.user_id = '51952841852@c.us'
#masami.user_id =  '51952841852@c.us'

In [104]:
hist.add_user(xiomara)

[0, '']

In [12]:
hist.get_count(xiomara)

[0, '']

In [13]:
hist.count(xiomara,count=False)

{'51934935842@c.us': [0, '']}

In [14]:
def start_menu(user:User) ->str:
    menu = data["menu"]
    if user.message == "/start":
        hist.count(user)
        return menu["message"]
    else:
        None

def location(user:User) ->str:
    location = data["location"]
    if str(hist.get_count(user)) == location['layer']:
        if user.message == location["value"]:
            hist.count(user)
            return location["message"] 
    else:
        None

def location_surco(user:User) -> str:
    location_surco = data["location_surco"]
    if hist.get_count(user) == int(location_surco['layer']):
        if user.message == location_surco["value"]:
            hist.count(user)
            return location_surco["message"]
    else:
        None


In [160]:
class SendText:

    def __init__(self,user:User) -> None:
        self.user = user

    def __add_direccion(self) -> None:
        direccion_dict = {
        "r" : ("1","si"),
        "l" : ("2","no")
        }
        for key in direccion_dict.keys():
            if self.user.message != "0":
                if self.user.message in direccion_dict[key]:
                    hist.count(self.user,key)
                    print(1,hist.get_count(self.user))
                    return None
                else:
                    hist.count(self.user)
                    print(2,hist.get_count(self.user))
                    return None
            else:
                hist.count(self.user,reset=True)
                print(3,hist.get_count(self.user))
                hist.count(self.user)
                return None

    def tree_desicion(self) -> str:
        data = json.loads(open('./respuesta.json',mode="r",encoding="utf-8").read())
        
        for cat in data.keys():
            category = data[cat]
            if self.user.message == "0":
                hist.count(self.user,reset = True)
                
            if str(hist.get_count(self.user)[0]) == category["layer"]:
                if str(hist.get_count(self.user)[1] == category["direccion"]):
                    if self.user.message in category["value"]:
                        self.__add_direccion()
                        return category["message"]

In [161]:
print({"id":xiomara.user_id,
"message":xiomara.message})


{'id': '51934935842@c.us', 'message': '1'}


In [162]:
hist.count(xiomara,reset = True)

{'51934935842@c.us': [0, ''],
 '51934935843@c.us': [0, ''],
 '51952841852@c.us': [0, '']}

In [30]:
hist.count(xiomara,'r')

{'51934935842@c.us': [4, 'rrrr']}

In [165]:
xiomara.message = "1"
xiomara.user_id = "51934935842@c.us"

In [113]:
sendText = SendText(xiomara)


In [166]:
print(sendText.tree_desicion())

None


In [159]:
hist.count(xiomara)

{'51934935842@c.us': [4, 'r'],
 '51934935843@c.us': [0, ''],
 '51952841852@c.us': [0, '']}

In [154]:
hist.get_count(xiomara)

[0, '']

In [1]:
from pydantic import BaseModel, Field
import json

class User(BaseModel):
    user_id : str = Field(
        ...,
        min_length=1,
        max_length=50,
        example='51952841852@.se'
    )
    message : str = Field(
        ...,
        example='/proceso de seleccion'
    )

xiomara =User
xiomara.user_id = "51934935843@c.us"

class Historial:

    def __init__(self) -> None:
        self.dic = {}
    
    def add_user(self,user:User) -> dict:
        self.dic[user.user_id] = [0,""]
        return self.dic[user.user_id]

    def count(self,user:User,direccion:str = "" ,count=True,reset:bool =False) -> dict: 
        if count:
            self.dic[user.user_id][0] += 1
        self.dic[user.user_id][1] += direccion
        if reset:
            self.dic[user.user_id][0] = 0
            self.dic[user.user_id][1] = ""
        return self.dic
    
    def get_count(self, user:User) -> dict:
        return self.dic[user.user_id]

hist = Historial()

class SendText:

    def __init__(self,user:User,hist:Historial) -> None:
        self.user = user
        self.hist = hist
        hist.add_user(self.user)


    def __add_direccion(self) -> None:
        direccion_dict = {
        "l" : ("1","si"),
        "r" : ("2","no")
        }
        for key in direccion_dict.keys():
            if self.user.message != "0":
                if self.user.message in direccion_dict[key]:
                    self.hist.count(self.user,key)
                    return None
                else:
                    self.hist.count(self.user)
                    return None
            else:
                self.hist.count(self.user,reset=True)
                self.hist.count(self.user)
                return None

    def tree_desicion(self) -> str:
        data = json.loads(open('./respuesta.json',mode="r",encoding="utf-8").read())
        self.__add_direccion()
        print("debug",self.hist.get_count(self.user))
        for cat in data.keys():
            category = data[cat]
            if self.user.message == "0":
                self.hist.count(self.user,reset = True)
                self.hist.count(self.user)

            if str(self.hist.get_count(self.user)[0]) == category["layer"]:
                if str(self.hist.get_count(self.user)[1] == category["direccion"]):
                    if self.user.message in category["value"]:
                        return category["message"]

In [41]:
hist.count(xiomara,"l")

{'51934935843@c.us': [2, 'll']}

In [42]:
xiomara.message = "1"

print({"id":xiomara.user_id,
"message":xiomara.message})
print(hist.get_count(xiomara))

sendText = SendText(xiomara,hist)
print(sendText.tree_desicion())

{'id': '51934935843@c.us', 'message': '1'}
[2, 'll']
2 [1, 'l']
debug [1, 'l']
[1, 'l']
Contamos con dos locales:

1. *Surco*: Martín Pizarro 172 Urb. Valle Hermoso, Surco, Lima 33 - Perú
2. *Madgalena*: Av. Pershing 910, Magdalena, Lima 17 - Perú
